![NYC Skyline](image/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?

### Importing modules

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

### Importing datasets

In [2]:
prices=pd.read_csv("datasets/airbnb_price.csv")
xls=pd.read_excel("datasets/airbnb_room_type.xlsx")
room_types=pd.read_excel("datasets/airbnb_room_type.xlsx",sheet_name=0)
reviews=pd.read_csv("datasets/airbnb_last_review.tsv",sep="\t")

### Early analysis

In [3]:
prices.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [4]:
xls.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [5]:
reviews.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


> We see that the price column of prices dataframe contains of strings i.e. 124 dollars etc.In order to calculate the average price we need to operate on the prices.Hence , we have to remove the trailing string (dollars) and change the type to int.

In [6]:
prices['price']=prices['price'].str.extract('(\d+)')
prices['price']=prices['price'].astype("int")

### Analysing the price column of prices df

In [7]:
prices.describe()

,listing_id,price
count,2.520900e+04,25209.000000
mean,2.068922e+07,141.777936
std,1.102928e+07,147.349137
min,2.595000e+03,0.000000
25%,1.202273e+07,69.000000
50%,2.234391e+07,105.000000
75%,3.037669e+07,175.000000
max,3.645581e+07,7500.000000


### Removing the outliers

In [8]:
prices=prices[(prices.price !=0) & (prices.price !=7500)]

### Finding average prices 

In [9]:
avg_price=prices['price'].mean().round(2)


### Calculating price per month

In [10]:
price_per_month=prices['price']*365/12

In [11]:
prices['price_per_month']=price_per_month


### Finding average price per month

In [12]:
average_price_per_month=prices['price_per_month'].mean().round(2)

###### According to Zumper, a 1 bedroom apartment in New York City costs, on average 3100 per month.Finding how much more, in dollars, the average cost of an Airbnb listing is versus the private market, and store it as difference, rounding your final answer to two decimal places.

In [13]:
difference=prices['price_per_month']-3100

In [14]:
xls['room_type']

0        Entire home/apt
1        Entire home/apt
2        Entire home/apt
3           private room
4        Entire home/apt
              ...       
25204       PRIVATE ROOM
25205       PRIVATE ROOM
25206       Private room
25207    Entire home/apt
25208       Private room
Name: room_type, Length: 25209, dtype: object

### Removing the variations

In [15]:
xls['room_type']=xls['room_type'].str.lower()
xls['room_type']=xls['room_type'].convert_dtypes()


### Storing count of values for room types 

In [16]:
room_frequencies=xls['room_type'].value_counts()

### Analysing reviews dataframe

In [17]:
reviews

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019
...,...,...,...
25204,36425863,Rusaa,July 07 2019
25205,36427429,H Ai,July 07 2019
25206,36438336,Ben,July 07 2019
25207,36442252,Blaine,July 07 2019


### Converting to datetime

In [18]:
reviews['last_review']=pd.to_datetime(reviews['last_review'])


### Finding oldest and latest reviews

In [19]:
first_reviewed=reviews['last_review'].min()
last_reviewed=reviews['last_review'].max()

### Merging df for further analysis

In [20]:
rooms_and_prices=pd.merge(prices,room_types,on=['listing_id'])
airbnb_merged=pd.merge(rooms_and_prices,reviews,on=['listing_id'])

### Dropping missing values

In [21]:
airbnb_merged.dropna(inplace=True)
airbnb_merged


,listing_id,price,nbhood_full,price_per_month,description,room_type,host_name,last_review
0,2595,225,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,Entire home/apt,Jennifer,2019-05-21
1,3831,89,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,2019-07-05
2,5099,200,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,2019-06-22
3,5178,79,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,2019-06-09
...,...,...,...,...,...,...,...,...
25196,36425863,129,"Manhattan, Upper East Side",3923.750000,Lovely Privet Bedroom with Privet Restroom,PRIVATE ROOM,Rusaa,2019-07-07
25197,36427429,45,"Queens, Flushing",1368.750000,No.2 with queen size bed,PRIVATE ROOM,H Ai,2019-07-07
25198,36438336,235,"Staten Island, Great Kills",7147.916667,Seas The Moment,Private room,Ben,2019-07-07
25199,36442252,100,"Bronx, Mott Haven",3041.666667,1B-1B apartment near by Metro,Entire home/apt,Blaine,2019-07-07


### checking for duplicate values

In [22]:
airbnb_merged.duplicated().sum()

0

### Analyzing listing prices

In [23]:
borough=airbnb_merged["nbhood_full"].str.partition(",")[0]
airbnb_merged['borough']=borough
# print(borough)
boroughs=airbnb_merged.groupby(airbnb_merged['borough']).agg(['sum','mean','median','count'])
boroughs=boroughs.round(2)

/tmp/ipykernel_8757/1453238703.py:4: FutureWarning: ['nbhood_full', 'description', 'room_type', 'host_name', 'last_review'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  boroughs=airbnb_merged.groupby(airbnb_merged['borough']).agg(['sum','mean','median','count'])


### Sorting value by mean

In [24]:
boroughs=boroughs.sort_values(by=("price_per_month", "mean"),ascending=False)

### Creating pricing range by boroughs

In [25]:
label_names=['Budget','Average','Expensive','Extravagant']
ranges=[0,69,175,350,np.inf]
airbnb_merged['price_range']=pd.cut(airbnb_merged['price_per_month'],ranges,labels=label_names,include_lowest=True)
prices_by_borough=airbnb_merged.groupby(['borough','price_range']).agg(['count'])

### Creating dictionary with answers

In [26]:
airbnb_analysis={
    "avg_price":avg_price,
    "average_price_per_month":average_price_per_month,
    "difference":difference,
    "room_frequencies":room_frequencies,
    "first_reviewed":first_reviewed,
    "last_reviewed":last_reviewed,
    "prices_by_borough":prices_by_borough,
}

In [27]:
airbnb_analysis

{'avg_price': 141.53,
 'average_price_per_month': 4304.73,
 'difference': 0        3743.750000
 1        -392.916667
 2        2983.333333
 3        -697.083333
 4        1462.500000
             ...     
 25204     823.750000
 25205   -1731.250000
 25206    4047.916667
 25207     -58.333333
 25208   -2187.500000
 Name: price_per_month, Length: 25201, dtype: float64,
 'room_frequencies': entire home/apt    13266
 private room       11356
 shared room          587
 Name: room_type, dtype: Int64,
 'first_reviewed': Timestamp('2019-01-01 00:00:00'),
 'last_reviewed': Timestamp('2019-07-09 00:00:00'),
 'prices_by_borough':                           listing_id  price nbhood_full price_per_month  \
                                count  count       count           count   
 borough       price_range                                                 
 Bronx         Budget               0      0           0               0   
               Average              0      0           0              